**Importing required libraries**

In [1]:
import pandas as pd
import numpy as np
import folium
import geocoder
from geopy.geocoders import Nominatim
import json
from bs4 import BeautifulSoup
import requests
from sklearn.cluster import KMeans
import matplotlib.cm as cm 
import matplotlib.colors as colors 
from pandas.io.json import json_normalize
import warnings

warnings.filterwarnings('ignore')
print('done')

done


**List of Popular Cities including Mumbai(India). Goal is to get the Popular Cities which have different life styles compared to Mumbai,India for our adventurous tour**

**For this the cities we have selected are Colombo(Srilanka),Cape Town(SouthAfrica), Roma(Italy), Canberra(Australia), Santiago(Chile), Copenhagen(Denmark),Paris(France),Stockholm(Sweden),Mumbai(India),Moscow(Russia),Kuala-lumpur(Malaysia) and Male(Maldives)**

In [2]:
Places_List = ['Colombo, WP','Cape Town, WC','Roma, RM','Canberra, ACT','Santiago, RM','Copenhagen, DK','Paris, IDF','Stockholm, SE-AB', 'Mumbai, MH','Moscow, MO','Kuala lumpur, KL','Male, MLE']
Places = []
Longitude = []
Latitude = []
geolocator = Nominatim()

for address in Places_List:
    location = geolocator.geocode(address)
    
    Places.append(address.split(',')[0])
    Latitude.append(location.latitude)
    Longitude.append(location.longitude)

print('Extraction of Latitudes and Longitudes is completed')

Extraction of Latitudes and Longitudes is completed


In [3]:
places_data = pd.DataFrame()
places_data['City'] = Places
places_data['Longitude'] = Longitude
places_data['Latitude'] = Latitude
places_data.shape

(12, 3)

In [4]:
places_data = places_data.sort_values(by=['City'])
places_data

,City,Longitude,Latitude
3,Canberra,149.101268,-35.297591
1,Cape Town,18.417396,-33.928992
0,Colombo,79.907602,6.884788
5,Copenhagen,12.570072,55.686724
10,Kuala lumpur,101.701336,3.148157
11,Male,73.529046,4.191279
9,Moscow,37.617406,55.757028
8,Mumbai,72.835335,18.938771
6,Paris,2.334428,48.837081
2,Roma,12.485338,41.894802


**Inspired from the Coursera lab**

In [5]:
CLIENT_ID = 'SO1C150DV4X5TTIF0STVVODGYUDAFV3ZICKZQXAHECMZV41K' # your Foursquare ID
CLIENT_SECRET = 'UNS3NE2PQC5YCXSEH5TBWVRGCRZ5IVIT4B4ATLO2WOEXL0L3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
radius = 500


def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = None
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        if results is None:
            print('None result is :',name)
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    #nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues = pd.DataFrame(columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category'])

    for venue_list in venues_list:
        for item in venue_list:
            dict_ = {'City':item[0],
                  'City Latitude':item[1] , 
                  'City Longitude':item[2], 
                  'Venue':item[3], 
                  'Venue Latitude':item[4], 
                  'Venue Longitude':item[5], 
                  'Venue Category':item[6]
                
            }
            nearby_venues = nearby_venues.append(dict_,ignore_index=True)
            
    
    return(nearby_venues)

**Extracting Neighborhood Data from foursquare for all the given Capitals**

In [6]:
total_venues = getNearbyVenues(names=places_data['City'],
                                   latitudes=places_data['Latitude'],
                                   longitudes=places_data['Longitude']
                                  )
total_venues.head()

Canberra
Cape Town
Colombo
Copenhagen
Kuala lumpur
Male
Moscow
Mumbai
Paris
Roma
Santiago
Stockholm


,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Canberra,-35.297591,149.101268,Rowing Australia,-35.298409,149.099242,Athletics & Sports
1,Canberra,-35.297591,149.101268,ANU Sailing Club,-35.300244,149.100742,Harbor / Marina
2,Canberra,-35.297591,149.101268,Yarralumla Bay,-35.299564,149.105081,Beach
3,Cape Town,-33.928992,18.417396,South African National Art Gallery,-33.928833,18.416820,Art Museum
4,Cape Town,-33.928992,18.417396,South African Jewish Museum,-33.930046,18.416916,Museum


In [7]:
total_venues.shape

(677, 7)

**Now converting Venue Category into one hot encoding**

In [8]:
# one hot encoding
total_onehot = pd.get_dummies(total_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
total_onehot['City'] = total_venues['City'] 
# move neighborhood column to the first column
fixed_columns = ['City'] + list(col for col in total_onehot.columns if col != 'City')

total_onehot = total_onehot[fixed_columns]

total_onehot.head()

,City,Accessories Store,African Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Terminal,American Restaurant,Antique Shop,Arcade,...,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Watch Shop,Wine Bar,Yoga Studio
0,Canberra,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Canberra,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Canberra,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Cape Town,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Cape Town,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**Now Grouping by City on average number of venues for given venue categories. This data will be used for Clustering**

In [9]:
total_grouped = total_onehot.groupby(['City']).mean().reset_index()
print(total_grouped.shape)
total_grouped.head()

(12, 171)


,City,Accessories Store,African Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Terminal,American Restaurant,Antique Shop,Arcade,...,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Watch Shop,Wine Bar,Yoga Studio
0,Canberra,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
1,Cape Town,0.0,0.014706,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.014706,0.0,0.0,0.000000,0.0
2,Colombo,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
3,Copenhagen,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.013889,0.0,0.0,0.027778,0.0
4,Kuala lumpur,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0


**Now Using KMeans clustering we divide data into 2 clusters. Since our destination City life style should be contrast to that of Mumbai(looking for new and adventurous), we will check the list of cities in cluster in which Mumbai is NOT PART OF. And those cities will be our choice of adventurous tour**

In [10]:
kclusters = 2
total_grouped_cluster = total_grouped.drop(['City'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=99).fit(total_grouped_cluster)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [11]:
final_data = places_data

# add clustering labels
final_data['Cluster Labels'] = kmeans.labels_
final_data

,City,Longitude,Latitude,Cluster Labels
3,Canberra,149.101268,-35.297591,1
1,Cape Town,18.417396,-33.928992,0
0,Colombo,79.907602,6.884788,0
5,Copenhagen,12.570072,55.686724,0
10,Kuala lumpur,101.701336,3.148157,0
11,Male,73.529046,4.191279,0
9,Moscow,37.617406,55.757028,0
8,Mumbai,72.835335,18.938771,0
6,Paris,2.334428,48.837081,0
2,Roma,12.485338,41.894802,0


**So by using Kmeans clustering , its Canberra Australia life style that stood out compared to all other cities including Mumbai. So for our adventurous trip we will be heading to Canberra Australia :) **

**Now Mapping them on World Map**

In [12]:
#create map
#Zoom is 10 as 11 is missing few clusters in the image
map_clusters = folium.Map(location=[0, 0], zoom_start=2)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(final_data['Latitude'], final_data['Longitude'], final_data['City'], final_data['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [17]:
# Different Venues present in Canberra
total_venues[total_venues['City']=='Canberra']

,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Canberra,-35.297591,149.101268,Rowing Australia,-35.298409,149.099242,Athletics & Sports
1,Canberra,-35.297591,149.101268,ANU Sailing Club,-35.300244,149.100742,Harbor / Marina
2,Canberra,-35.297591,149.101268,Yarralumla Bay,-35.299564,149.105081,Beach
